In [ ]:
# arena_util.py
# -*- coding: utf-8 -*-
import io
import os
import json
from collections import Counter
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from collections import Counter # 컨테이너에 동일한 자료가 몇개인지 파악 : 출력값은 딕셔너리
import numpy as np
import pandas as pd
import scipy.sparse as spr # Sparse matrices 출력
import pickle

# json 파일 작성
def write_json(data, fname):
  def _conv(o):
    # isinstance : 주어진 인스턴스가 특정 클래스/데이터 타입인지 확인
    if isinstance(o, np.int64) or isinstance(o, np.int32):
      return int(o) # int64, int32인지 확인 후 int로 출력
    raise TypeError
  
  parent = os.path.dirname(fname) # os.path.dirname(fname) : 경로 중 디렉토리 이름까지만 얻기
  distutils.dir_util.mkpath("C:/Users/이다은/Desktop/BOAZ 14기/추천시스템/" + parent) # 디렉토리 만들기

  # 파일 입출력( W : 쓰기)
  with io.open("C:/Users/이다은/Desktop/BOAZ 14기/추천시스템/" + fname, "w", encoding="utf8") as f:
    # json.dumps : 직렬화해서 객체를 파일에 쓴다
    json_str = json.dumps(data, ensure_ascii=False, default = _conv)
    f.write(json_str)

# json 파일 로드
def load_json(fname):
  with open(fname, encoding='utf8') as f :
    json_obj = json.load(f)

  return json_obj

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
val = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/val.json', typ = 'frame',encoding='UTF-8')
song_meta = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/song_meta.json', typ = 'frame',encoding='UTF-8')
train = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/train.json', typ = 'frame', encoding='utf-8')
test = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/test.json', typ = 'frame', encoding='utf-8')
genre_gn_all = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/genre_gn_all.json', typ = 'series',encoding='utf-8')

In [ ]:
n_train = len(train)
n_val = len(val)
print('train의 개수는 {}개 , val의 개수는 {}개, test의 개수는 {}개! '.format(len(train), len(val), len(test)))

train의 개수는 115071개 , val의 개수는 23015개, test의 개수는 10740개! 


In [ ]:
# num 열 추가
train.loc[:, 'num_songs'] = train['songs'].map(len)
train.loc[:, 'num_tags'] = train['tags'].map(len)
val.loc[:, 'num_songs'] = val['songs'].map(len)
val.loc[:, 'num_tags'] = val['tags'].map(len)

In [ ]:
print(train['num_songs'].sum(), val['num_songs'].sum())

5285871 421199


## train 사용 곡 수 줄이기
1. 10번 이상 등장한 것만 남김
2. 날짜 이상한거 삭제
3. 장르 없는거 삭제


In [ ]:
%%time
## 1. 10번이상 등장한 노래만 사용
song_count = []
for i in train.songs.to_list():
  for j in i:
    song_count.append(j)
song = pd.DataFrame(song_count)[0].value_counts().reset_index().rename(columns={'index':'song', 0:'count'})

# 10번 이상 등장한 노래만 사용(9만개)
song_value = song.query('count>9').song.values
def song_del(x):
  x_iter = []
  for i in x:
    if i in song_value:
      x_iter.append(i)
  return x_iter

train['songs'] = train.songs.map(song_del)

CPU times: user 2min 46s, sys: 12.6 ms, total: 2min 46s
Wall time: 2min 46s


In [ ]:
## 2. 이상한 날짜 제거
plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest

plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)
strange_songs = strange_songs.drop_duplicates(['id'])

date_id = strange_songs.id.values

def date_del(x):
  x_iter = []
  for i in x:
    if i not in date_id:
      x_iter.append(i)
  return x_iter

train['songs'] = train.songs.map(date_del)

In [ ]:
# ## 3. 장르 없는거 제거
# %%time
# def list_to_nan(x) : 
#     if x == [] : 
#         return (np.nan)
#     else : return x

# song_meta['song_gn_gnr_basket'] = song_meta['song_gn_gnr_basket'].apply(list_to_nan)
# song_meta['song_gn_gnr_basket'] = song_meta['song_gn_gnr_basket'].fillna('#')
# no_gnr = song_meta[song_meta['song_gn_gnr_basket']=='#'] # 대분류 장르 없는 1059개 song

# no_gnr_list = no_gnr.id.values # str

# def gnr_del(x):
#   x_iter = []
#   for i in x:
#     if i not in no_gnr_list:
#       x_iter.append(i)
#   return x_iter

# train['songs'] = train.songs.map(gnr_del)

In [ ]:
# 몇개나 줄었게? 백만개!
train.loc[:, 'num_songs'] = train['songs'].map(len)
print(train['num_songs'].sum()) 

3990654


## id 재생성

In [ ]:
### plylst id 재생성

train['istrain'] = 1
val['istrain'] = 0

n_train = len(train)
n_val = len(val)

# train+val = plylst
plylst = pd.concat([train, val], ignore_index = True) 
# ignore_index = True : 열 이름 무시하고 정수 번호 자동 부여

## train과 val 합침!! 
plylst['nid'] = range(n_train + n_val) # 인덱스 재생성

# plylst의 id를 nid로 순서대로 바꾼다
# zip : 동일한 개수를 가진 자료형을 묶어 주는 역할 > zip으로 묶어서 딕셔너리로 만든다
plylst_id_nid = dict(zip(plylst['id'], plylst['nid']))
plylst_nid_id = dict(zip(plylst['nid'], plylst['id']))

plylst_tag = plylst['tags'] 
tag_counter = Counter( [tg for tgs in plylst_tag for tg in tgs] ) # 딕셔너리 형태로 출력
tag_dict = {x: tag_counter[x] for x in tag_counter} # Counter({딕셔너리}) 형태임 > 그냥 딕셔너리로 바꿔주기

# 태그 아이디 재생성
tag_id_tid = dict() # 딕셔너리 만들기
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
  tag_id_tid[t] = i
  tag_tid_id[i] = t

n_tags = len(tag_dict) # 태그 종류 개수

plylst_song = plylst['songs']
song_counter = Counter( [sg for sgs in plylst_song for sg in sgs] )
song_dict = {x : song_counter[x] for x in song_counter}

song_id_sid = dict()
song_sid_id = dict()
for i, t in enumerate(song_dict):
  song_id_sid[t] = i
  song_sid_id[i] = t

n_songs = len(song_dict) # song  종류 개수


In [ ]:
print(n_songs, n_tags, song_meta.shape) # song의 종류 개수 60만개 > 16만개로 줄음

166979 30197 (707989, 9)


In [ ]:
## 새로운 id 적용

plylst['songs_id'] = plylst['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None]) # 아이디로 리스트 만들어줌. 
plylst['tags_id'] = plylst['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])
plylst_use = plylst[['istrain', 'nid', 'updt_date', 'songs_id', 'tags_id']]
# num_songs, num_tags 열 추가
plylst_use.loc[:, 'num_songs'] = plylst_use['songs_id'].map(len)
plylst_use.loc[:, 'num_tags'] = plylst_use['tags_id'].map(len)
plylst_use = plylst_use.set_index('nid')
# 나누기
plylst_train = plylst_use.iloc[:n_train, :]
plylst_val = plylst_use.iloc[n_train:, :]

plylst_train.head(2)

,istrain,updt_date,songs_id,tags_id,num_songs,num_tags
nid,,,,,,
0,1,2013-12-19 18:36:19.000,"[0, 1, 2, 3, 4, 5]",[0],6,1
1,1,2014-12-02 16:19:42.000,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[1, 2]",33,2


## song-gnr-new_id 데이터프레임

In [ ]:
## 1. 원래 id와 new id
id = list(song_id_sid.keys())
new_id = list(song_id_sid.values())
# 딕셔너리 데이터프레임으로 만들기
new_song_df = pd.DataFrame(id, new_id).reset_index()
new_song_df = new_song_df.rename(columns={'index':'new_id', 0:'songs'})

In [ ]:
new_song_df

,new_id,songs
0,0,562083
1,1,297861
2,2,139541
3,3,650298
4,4,531057
...,...,...
166974,166974,328334
166975,166975,586564
166976,166976,444003
166977,166977,665833


In [ ]:
new_song_df[new_song_df['new_id']==83106]

,new_id,songs
83106,83106,336285


In [ ]:
song_meta[song_meta['id']==344058] # 왜 제거가 안됐지.......

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
344058,[],20200331,"아무렇지 않게, 안녕",10410478,[1229685],여행의 색깔 (Feat. 스무살),[],[HYNN (박혜원)],344058


In [ ]:
# ## 2. 원래 song id와 장르 
song_meta = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/song_meta.json', typ = 'frame',encoding='UTF-8')
# song_meta['song_gn_gnr_basket'] = song_meta['song_gn_gnr_basket'].apply(list_to_nan)
# song_meta['song_gn_gnr_basket'].isnull().sum()
# song_meta = song_meta.dropna() # 장르 없는노래 애초에 제거하고 시작함(환멸이 나서 안되겠다;;;)

In [ ]:
song_meta[song_meta['id']==143209]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
143209,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20150522,None,2318271,[726168],사랑바보,"[GN0500, GN0600]",[민은서밴드],143209


In [ ]:
## song_장르 matrix
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]
song_gnr_map['song_gn_gnr_basket'] = song_gnr_map.song_gn_gnr_basket.apply(lambda x: x if len(x) >= 1 else [None]) # 마법이다.....

# unnest song_gn_dtl_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'songs', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

In [ ]:
song_gnr_map # 모든 song에 대한 장르정보 데이터

,songs,gnr_code
0,0,GN0900
1,1,GN1600
2,2,GN0900
3,3,GN1100
4,4,GN1800
...,...,...
803913,707984,GN2000
803914,707985,GN0900
803915,707986,GN0100
803916,707987,GN1800


In [ ]:
# 두 데이터프레임 합치기
song_gnr_map['songs'] = song_gnr_map['songs'].astype(int)
gnr_song = pd.merge(song_gnr_map, new_song_df, on='songs').fillna('장르없음').sort_values(by='new_id').reset_index()
gnr_song.drop(['index'], axis=1, inplace=True)

In [ ]:
gnr_song

,songs,gnr_code,new_id
0,562083,GN1000,0
1,562083,GN0900,0
2,297861,GN0900,1
3,297861,GN1000,1
4,139541,GN1000,2
...,...,...,...
203422,328334,GN0800,166974
203423,586564,GN0100,166975
203424,444003,GN0100,166976
203425,665833,GN1600,166977


In [ ]:
gnr_song['new_id'] = gnr_song['new_id'].drop_duplicates()

In [ ]:
gnr_song = gnr_song.dropna()

In [ ]:
gnr_song

,songs,gnr_code,new_id
0,562083,GN1000,0.0
2,297861,GN0900,1.0
4,139541,GN1000,2.0
6,650298,GN0900,3.0
7,531057,GN1300,4.0
...,...,...,...
203422,328334,GN0800,166974.0
203423,586564,GN0100,166975.0
203424,444003,GN0100,166976.0
203425,665833,GN1600,166977.0


In [ ]:
f1 = pd.pivot_table(data = gnr_song,index='new_id',columns='gnr_code',aggfunc='count',fill_value=0, margins=True)
f1 = f1.iloc[:-1,:-1].div(f1.iloc[:-1,-1], axis=0).reset_index()
f1['new_id'] = f1['new_id'].map(lambda x : int(x))

In [ ]:
f1.drop(['new_id'], axis=1, inplace=True)

In [ ]:
f1 # 너무 sparse 한가..................

songs                              ...                                 
gnr_code GN0100 GN0200 GN0300 GN0400 GN0500  ... GN2700 GN2800 GN2900 GN9000 장르없음
0           0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
1           0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
2           0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
3           0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
4           0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
...         ...    ...    ...    ...    ...  ...    ...    ...    ...    ...  ...
166974      0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
166975      1.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
166976      1.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
166977      0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0
166978      0.0    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0    0.0  0.0

[166979 rows x 31 columns]

In [ ]:
f1_array = f1.values
f1_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
f1_array.shape

(166979, 31)

In [ ]:
f1_array_T = f1_array.T
f1_array_T.shape

(31, 166979)

## 장르-plylst 행렬

In [ ]:
val = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/val.json', typ = 'frame',encoding='UTF-8')
song_meta = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/song_meta.json', typ = 'frame',encoding='UTF-8')
train = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/train.json', typ = 'frame', encoding='utf-8')
test = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/test.json', typ = 'frame', encoding='utf-8')
genre_gn_all = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/genre_gn_all.json', typ = 'series',encoding='utf-8')

In [ ]:
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']

In [ ]:
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_dtl_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

In [ ]:
def list_to_nan(x) : 
    if x == [] : 
        return (np.nan)
    else : return x

train['songs'] = train['songs'].apply(list_to_nan)
train=train.dropna()
val['songs'] = val['songs'].apply(list_to_nan)
val=val.dropna()

In [ ]:
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [ ]:
plylst_song_map

,id,songs
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861
...,...,...
5285866,100389,111365
5285867,100389,51373
5285868,100389,640239
5285869,100389,13759


In [ ]:
song_gnr_map

,song_id,gnr_code
0,0,GN0900
1,1,GN1600
2,2,GN0900
3,3,GN1100
4,4,GN1800
...,...,...
802854,707984,GN2000
802855,707985,GN0900
802856,707986,GN0100
802857,707987,GN1800


In [ ]:
song_gnr_map.columns = ['songs','gmr_code']

train_g = pd.merge(plylst_song_map,song_gnr_map, how='left', on='songs').fillna('장르없음')

In [ ]:
train_g

,id,songs,gmr_code
0,61281,525514,GN1400
1,61281,129701,GN0900
2,61281,129701,GN1000
3,61281,383374,GN1000
4,61281,562083,GN0900
...,...,...,...
6837821,100389,51373,GN1300
6837822,100389,640239,GN1500
6837823,100389,13759,GN0900
6837824,100389,13759,GN1000


In [ ]:
f2 = pd.pivot_table(data = train_g,index='id',columns='gmr_code',aggfunc='count',fill_value=0,values='songs',margins=True)

In [ ]:
f2 = f2.iloc[:-1,:-1].div(f2.iloc[:-1,-1], axis=0).reset_index()

f2['id'] = f2['id'].map(lambda x : int(x))

In [ ]:
f2.drop(['id'], axis=1, inplace=True)

In [ ]:
f2

gmr_code,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,GN0900,GN1000,GN1100,GN1200,GN1300,GN1400,GN1500,GN1600,GN1700,GN1800,GN1900,GN2000,GN2100,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN9000,장르없음
0,0.000000,0.000000,0.714286,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000
1,0.052632,0.157895,0.157895,0.000000,0.000000,0.000000,0.052632,0.000000,0.157895,0.052632,0.0,0.157895,0.000000,0.052632,0.000000,0.000,0.000000,0.000000,0.000000,0.105263,0.0,0.0,0.0,0.0,0.052632,0.000000,0.0,0.0,0.0,0.0,0.000
2,0.148936,0.000000,0.000000,0.000000,0.319149,0.340426,0.000000,0.085106,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.085106,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.021277,0.0,0.0,0.0,0.0,0.000
3,0.089286,0.000000,0.035714,0.071429,0.107143,0.053571,0.000000,0.035714,0.000000,0.000000,0.0,0.017857,0.035714,0.053571,0.035714,0.000,0.071429,0.321429,0.071429,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000
4,0.590909,0.045455,0.000000,0.136364,0.090909,0.045455,0.000000,0.090909,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115066,0.072464,0.000000,0.000000,0.086957,0.362319,0.115942,0.000000,0.289855,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.028986,0.000,0.028986,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.014493,0.0,0.0,0.0,0.0,0.000
115067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.428571,0.238095,0.0,0.000000,0.333333,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000
115068,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.625000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.125
115069,0.016949,0.000000,0.677966,0.033898,0.271186,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000


In [ ]:
f2_array = f2.values
f2_array

array([[0.        , 0.        , 0.71428571, ..., 0.        , 0.        ,
        0.        ],
       [0.05263158, 0.15789474, 0.15789474, ..., 0.        , 0.        ,
        0.        ],
       [0.14893617, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.125     ],
       [0.01694915, 0.        , 0.6779661 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.28571429, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
f2_array.shape

(115071, 31)

In [ ]:
f2_array_T = f2_array.T
f2_array_T.shape

(31, 115071)

In [ ]:
# 입실론처리
f1_array = f1_array+0.1
f2_array = f2_array+0.1

## 모델링

In [ ]:
# 노래 행렬
row = np.repeat(range(n_train), plylst_train['num_songs']) # range(n_train)을 num_songs(각 플레이리스트의 노래 개수)만큼 반복! = 5285871개의 1차원 array
col = [song for songs in plylst_train['songs_id'] for song in songs] # 5285871개 (plylst에 등장한 모든 song = 중복 포함 )
dat = np.repeat(1, plylst_train['num_songs'].sum()) # row 와 col의 교차지점에 1을 박음. 

# 5285871 * 5285871 정사각행렬을 만들고 이를 csr_matrix를 활용해 바꿈. 

train_songs_A = spr.csr_matrix( (dat, (row, col) ), shape=(n_train, n_songs) ) # shape을 115071(train 개수) * 638336(song 개수)
train_songs_A

<115071x166979 sparse matrix of type '<class 'numpy.longlong'>'
	with 3990654 stored elements in Compressed Sparse Row format>

In [ ]:
# train_songs_A = f2_array.dot(f1_array_T)

In [ ]:
# 태그 행렬
row = np.repeat(range(n_train), plylst_train['num_tags']) # 476331개 : plylst별 태그 등장 횟수 반복
col = [tag for tags in plylst_train['tags_id'] for tag in tags] # 476331개 : 각 plylst별 태그(중복 포함)
# 행렬 안에 tag의 개수를 넣음 (빈도 수)
dat = np.repeat(1, plylst_train['num_tags'].sum()) # input array 는 한개
# np.repeat(input array, 요소 반복 횟수)
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_tags))
train_tags_A

<115071x30197 sparse matrix of type '<class 'numpy.longlong'>'
	with 476331 stored elements in Compressed Sparse Row format>

In [ ]:
## transpose ( 전치 )
train_songs_A_T = train_songs_A.T.tocsr() # tocar() : Compressed Sparse row로 만들어줌
train_tags_A_T = train_tags_A.T.tocsr() 

In [ ]:
print(f1_array.shape, f2_array.shape)

(166979, 31) (115071, 31)


In [ ]:
from tqdm import tqdm

def rec(pids):

  %%time
  res = []

  for pid in pids:
    p = np.zeros((n_songs,1)) # n_songs 만큼 0박은 1차원 array 만들기

    p[ plylst_val.loc[pid,'songs_id'] ] = 1 # 각 plylst에 이미 있는 song_id에는 1박기

    # 이미 있는 id
    songs_already = plylst_val.loc[pid, "songs_id"]
    tags_already = plylst_val.loc[pid, "tags_id"]
    
    # song 빈도수 열과 p(이미 존재하는 song id에 1 박은거) 내적! 
    valid = train_songs_A.dot(p).reshape(-1)
    # reshape(-1) : 다른 나머지 차원의 크기를 맞추고 남은 크기를 해당 차원에 할당

    # ## 장르 추가 # 악재;;
    gnr_add = f2_array_T.dot(valid)
    new = f2_array.dot(gnr_add)

    # 전치시킨거에 valid(위에서 내적한거) 또 내적
    cand_song = train_songs_A_T.dot(new)
    cand_song_idx = cand_song.reshape(-1).argsort()[-1000:][::-1] 
    # argsort() : 작은 값부터 순서대로 데이터 인덱스 반환
    # array[::-1] : 처음부터 끝까지 -1칸 간격으로(역순으로) > 큰값부터 반환
    # [-150:] : 150개 뒤에서부터 자름
    cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]
    # cand_song_idx 에서 songs_already 가 있는지 T/F로 알려준다.
    # 근데 False 만 골라서, 100개까지 자른다

    rec_song_idx = [song_sid_id[i] for i in cand_song_idx]
    # cand_song_idx 의 index의 i를 song_sid_id[i]로 가져온다. = 최종 제출본

######################################################################################
    cand_tag = train_tags_A_T.dot(valid)
    cand_tag_idx = cand_tag.reshape(-1).argsort()[-100:][::-1]
    # 뒤에서 큰 순서대로 15개 자름. 큰 순서대로 데이터 정렬
    cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
    # 이미 태그에 들어있지 않은 것만(False) 10개 반환
    rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]
    #cand_tag_idx 의 인덱스를 가져와서 rec_tag_idx에 넣는다

    res.append({
                "id": plylst_nid_id[pid],
                "songs": rec_song_idx,
                "tags": rec_tag_idx
            })
    # res에 딕셔너리 append
    
    # 1000번마다 시간 출력
    tt = 1
    if tt % 1000 == 0:
      print(tt)

    tt += 1
  return res  

In [ ]:
# 학습! 일해라 일해
%%time
answers = rec(plylst_val.index)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
CPU times: user 34min 49s, sys: 53min 52s, total: 1h 28min 42s
Wall time: 22min 20s


In [ ]:
import io
import os

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

write_json(answers,'/content/gdrive/My Drive/Colab Notebooks/머신/results5.json')

터짐 테스트..

In [ ]:
# p = np.zeros((n_songs,1)) # n_?songs 만큼 0박은 1차원 array 만들기

# p[ plylst_val.loc[1,'songs_id'] ] = 1 # 각 plylst에 이미 있는 song_id에는 1박기
    
#     # song 빈도수 열과 p(이미 존재하는 song id에 1 박은거) 내적! 
# valid = train_songs_A.dot(p).reshape(-1)
#     # reshape(-1) : 다른 나머지 차원의 크기를 맞추고 남은 크기를 해당 차원에 할당

#     # 이미 있는 id
# songs_already = plylst_val.loc[1, "songs_id"]
# tags_already = plylst_val.loc[1, "tags_id"]

#     # 전치시킨거에 valid(위에서 내적한거) 또 내적
# cand_song = train_songs_A_T.dot(valid)
# cand_song_idx = cand_song.reshape(-1).argsort()[-1000:][::-1] 
#     # argsort() : 작은 값부터 순서대로 데이터 인덱스 반환
#     # array[::-1] : 처음부터 끝까지 -1칸 간격으로(역순으로) > 큰값부터 반환
#     # [-150:] : 150개 뒤에서부터 자름
# cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]
#     # cand_song_idx 에서 songs_already 가 있는지 T/F로 알려준다.
#     # 근데 False 만 골라서, 100개까지 자른다

# rec_song_idx = [song_sid_id[i] for i in cand_song_idx]